In [ ]:
import pandas as pd

# Reading from a JSON file
df_brands = pd.read_json('brands.json', lines=True)
df = pd.json_normalize(df_brands.to_dict(orient='records'), sep='_')

# df['createdDate_$date'] = pd.to_datetime(df['createdDate_$date'], unit='ms')
# df['lastLogin_$date'] = pd.to_datetime(df['lastLogin_$date'], unit='ms')

df
# df_receipts = pd.read_json('receipts.json')
# df_users = pd.read_json('users.json')

# df_brands

# {
#     "_id": {
#         "$oid": "601ac115be37ce2ead437551"
#     },
#     "barcode": "511111019862",
#     "category": "Baking",
#     "categoryCode": "BAKING",
#     "cpg": {
#         "$id": {
#             "$oid": "601ac114be37ce2ead437550"
#         },
#         "$ref": "Cogs"
#     },
#     "name": "test brand @1612366101024",
#     "topBrand": false
# }


: 

In [77]:
import pandas as pd

# # Load and normalize users data
# users_df = pd.read_json('users.json', lines=True)
# users_df = pd.json_normalize(users_df.to_dict(orient='records'), sep='_')

# #Display the data
# # print("Users Data:")
# # users_df


#Load and normalize brands data
# brands_df = pd.read_json('brands.json', lines=True)
# brands_df = pd.json_normalize(brands_df.to_dict(orient='records'), sep='_')

# print("\nbrands Data:")
# brands_df


# with open("receipts.json", "r", encoding="utf-8") as file:
#     first_lines = [next(file) for _ in range(5)]  # Read first 5 lines
#     print(first_lines)


# # Load and normalize receipt data
receipts_df = pd.read_json('receipts.json', lines=True)
receipts_df = pd.json_normalize(receipts_df.to_dict(orient='records'), sep='_')
print("\nReceipts Data:")
receipts_df

# print(receipts_df.head())  # Preview the flattened DataFrame


# Expand 'rewardsReceiptItemList' into separate rows
receipt_items_df = receipts_df.explode('rewardsReceiptItemList')
receipt_items_df = pd.json_normalize(receipt_items_df['rewardsReceiptItemList'])

# # Normalize nested fields
# df = pd.json_normalize(
#     receipts_df, 
#     record_path=['rewardsReceiptItemList'],  # Extract nested list
#     meta=['_id', 'userId', 'totalSpent']  # Keep parent-level fields
# )


#Display the data
# print("Users Data:")
# users_df


print("\nReceipt Items Data:")
print(receipt_items_df.head())



Receipts Data:

Receipt Items Data:
        barcode                                        description finalPrice  \
0          4011                                     ITEM NOT FOUND      26.00   
1          4011                                     ITEM NOT FOUND          1   
2  028400642255  DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...      10.00   
3           NaN                                                NaN        NaN   
4          4011                                     ITEM NOT FOUND      28.00   

  itemPrice needsFetchReview partnerItemId preventTargetGapPoints  \
0     26.00            False             1                   True   
1         1              NaN             1                    NaN   
2     10.00             True             2                   True   
3       NaN            False             1                   True   
4     28.00            False             1                   True   

   quantityPurchased userFlaggedBarcode userFlaggedNewItem  .

In [ ]:
import pandas as pd
import json

# # Load and normalize users data
# users_df = pd.read_json('users.json', lines=True)
# users_df = pd.json_normalize(users_df.to_dict(orient='records'), sep='_')

# # Load and normalize brands data
# brands_df = pd.read_json('brands.json', lines=True)
# brands_df = pd.json_normalize(brands_df.to_dict(orient='records'), sep='_')


# # Load and normalize receipt data
# receipts_df = pd.read_json('receipts.json', lines=True)
# receipts_df = pd.json_normalize(receipts_df.to_dict(orient='records'), sep='_')

# # Expand 'rewardsReceiptItemList' into separate rows
# receipt_items_df = receipts_df.explode('rewardsReceiptItemList')
# receipt_items_df = pd.json_normalize(receipt_items_df['rewardsReceiptItemList'])

# # Display the data
# print("Users Data:")
# print(users_df.head())

# print("\nbrands Data:")
# print(brands_df.head())

# print("\nReceipts Data:")
# print(receipts_df.head())

# print("\nReceipt Items Data:")
# print(receipt_items_df.head())


# Load receipts data
with open("receipts.json", "r") as file:
    receipts_data = [json.loads(line) for line in file]

receipts_df = pd.DataFrame(receipts_data)

# Load brands data
with open("brands.json", "r") as file:
    brands_data = [json.loads(line) for line in file]

brands_df = pd.DataFrame(brands_data)

# Convert `dateScanned` to datetime
receipts_df["dateScanned"] = pd.to_datetime(receipts_df["dateScanned"].apply(lambda x: x["$date"]), unit="ms")

# Get the most recent month
most_recent_month = receipts_df["dateScanned"].max().strftime("%Y-%m")

# Filter receipts for the most recent month
filtered_receipts = receipts_df[receipts_df["dateScanned"].dt.strftime("%Y-%m") == most_recent_month]

# Extract brand counts from the receipts
brand_counts = filtered_receipts.explode("rewardsReceiptItemList")  # Expand list items
brand_counts["brandId"] = brand_counts["rewardsReceiptItemList"].apply(lambda x: x.get("rewardsProductPartnerId") if isinstance(x, dict) else None)

# Count receipts per brand
top_brands = brand_counts["brandId"].value_counts().head(5).reset_index()
top_brands.columns = ["brandId", "receipt_count"]

# Merge with brand names
top_brands = top_brands.merge(brands_df[["_id", "brandName"]], left_on="brandId", right_on="_id", how="left")

# Select relevant columns
top_brands = top_brands[["brandName", "receipt_count"]]

print(top_brands)



Receipt Items Data:
        barcode                                        description finalPrice  \
0          4011                                     ITEM NOT FOUND      26.00   
1          4011                                     ITEM NOT FOUND          1   
2  028400642255  DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...      10.00   
3           NaN                                                NaN        NaN   
4          4011                                     ITEM NOT FOUND      28.00   

  itemPrice needsFetchReview partnerItemId preventTargetGapPoints  \
0     26.00            False             1                   True   
1         1              NaN             1                    NaN   
2     10.00             True             2                   True   
3       NaN            False             1                   True   
4     28.00            False             1                   True   

   quantityPurchased userFlaggedBarcode userFlaggedNewItem  ... itemNumber  \

In [72]:
import json
import pandas as pd


df = pd.read_json("receipts.json", lines=True)
print(df.head())

# # Read the raw JSON file
# with open("receipts.json", "r", encoding="utf-8") as file:
#     data = json.load(file)  # Load the JSON file as a dictionary/list

# # Print the first item to inspect its structure
# print(json.dumps(data[0], indent=2))

# # Function to clean MongoDB-style JSON
# def clean_mongo_json(obj):
#     if isinstance(obj, dict):  # If it's a dictionary
#         if "$oid" in obj:  # Convert MongoDB ObjectId to a string
#             return obj["$oid"]
#         elif "$date" in obj:  # Convert MongoDB Date to a readable format
#             return pd.to_datetime(obj["$date"], unit="ms")  # Convert timestamp to datetime
#         else:
#             return {k: clean_mongo_json(v) for k, v in obj.items()}  # Recursively clean
#     elif isinstance(obj, list):  # If it's a list, clean each element
#         return [clean_mongo_json(i) for i in obj]
#     else:
#         return obj  # Return as is if it's not a dict or list

# # Apply cleaning function to the entire dataset
# cleaned_data = clean_mongo_json(data)



# # Convert cleaned JSON to DataFrame
# df = pd.DataFrame(cleaned_data)

# # Print sample rows
# print(df.head())




                                    _id  bonusPointsEarned  \
0  {'$oid': '5ff1e1eb0a720f0523000575'}              500.0   
1  {'$oid': '5ff1e1bb0a720f052300056b'}              150.0   
2  {'$oid': '5ff1e1f10a720f052300057a'}                5.0   
3  {'$oid': '5ff1e1ee0a7214ada100056f'}                5.0   
4  {'$oid': '5ff1e1d20a7214ada1000561'}                5.0   

                             bonusPointsEarnedReason  \
0  Receipt number 2 completed, bonus point schedu...   
1  Receipt number 5 completed, bonus point schedu...   
2                         All-receipts receipt bonus   
3                         All-receipts receipt bonus   
4                         All-receipts receipt bonus   

                 createDate               dateScanned  \
0  {'$date': 1609687531000}  {'$date': 1609687531000}   
1  {'$date': 1609687483000}  {'$date': 1609687483000}   
2  {'$date': 1609687537000}  {'$date': 1609687537000}   
3  {'$date': 1609687534000}  {'$date': 1609687534000}   
4  {'

In [52]:
import pandas as pd
import json

# Load JSON file as line-delimited JSON
file_path = "receipts.json"

with open(file_path, "r", encoding="utf-8") as file:
    data = [json.loads(line) for line in file]  # Read line by line

# Convert to DataFrame
df = pd.DataFrame(data)

# Print first few rows
print(df.head())


JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 1 (char 9)

In [57]:
import json
import re
from datetime import datetime

# Function to replace single quotes with double quotes and fix other issues
def preprocess_json(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as infile:
        data = infile.read()
        
    # Replace single quotes with double quotes for keys and string values
    data = re.sub(r"(?<=[:\[,])\s*'(.*?)'\s*(?=\s*[:,\]\}])", r'"\1"', data)
    # Ensure double quotes for the keys and string values, and remove any extra commas at the end of objects/arrays
    data = re.sub(r"(?<=[:\[,])\s*'(.*?)'\s*(?=\s*[:,\]\}])", r'"\1"', data)
    
    return data

# Function to convert MongoDB-specific fields to standard JSON format
def convert_mongodb_to_standard(json_data):
    # Recursively process the data
    if isinstance(json_data, dict):
        # If the dictionary contains "$oid", convert it to a string
        if "$oid" in json_data:
            return str(json_data["$oid"])
        # If the dictionary contains "$date", convert it to ISO 8601 format
        if "$date" in json_data:
            return datetime.utcfromtimestamp(json_data["$date"] / 1000).strftime('%Y-%m-%dT%H:%M:%SZ')
        # Process each key-value pair recursively
        return {key: convert_mongodb_to_standard(value) for key, value in json_data.items()}
    elif isinstance(json_data, list):
        # If it's a list, recursively process each item
        return [convert_mongodb_to_standard(item) for item in json_data]
    else:
        # Return the data as is if it's not a dictionary or list
        return json_data

# Function to process the entire JSON file
def process_json_file(input_file_path, output_file_path):
    # Preprocess the input file to ensure valid JSON format
    processed_data = preprocess_json(input_file_path)
    
    # Load the data from the preprocessed content
    data = json.loads(processed_data)
    
    # Convert each record in the data to the standard JSON format
    converted_data = [convert_mongodb_to_standard(record) for record in data]
    
    # Write the converted data to the output JSON file
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        json.dump(converted_data, outfile, indent=4)

# Input and output file paths
input_file_path = 'receipts.json'
output_file_path = 'receipts_converted.json'

# Process the file
process_json_file(input_file_path, output_file_path)

print("Conversion completed. The output is saved in 'receipts_converted.json'.")


JSONDecodeError: Extra data: line 16 column 1 (char 941)